# Lab 2: Image Enhancement

In [6]:
import cv2
import os, numpy as np
from matplotlib import pyplot as plt

# Path to directory with images
dataDir = './Images_03a'

In [125]:
# Open noisy image
img = cv2.imread(os.path.join(dataDir, 'coins_03_noisy.jpg'))

# Show image
cv2.imshow('image', img)
cv2.waitKey(0)
cv2.destroyWindow('image')

### 1. Filtering and Smoothing

[Check tutorial here!](https://docs.opencv.org/4.x/dc/dd3/tutorial_gausian_median_blur_bilateral_filter.html)

In [18]:
# Apply mean filter to the image
img_mean_filter = cv2.blur(img, (4, 4))

# Show image
cv2.imshow('image', img_mean_filter)
cv2.waitKey(0)
cv2.destroyWindow('image')

In [32]:
# Apply a Gaussian filter to the image
img_gaussian_filter = cv2.GaussianBlur(img, (5,5), 0)

# Show image
cv2.imshow('image', img_gaussian_filter)
cv2.waitKey(0)
cv2.destroyWindow('image')

Exercise 1.1: Apply median and bilateral filters to the image

In [42]:
# Apply a median filter
img_median_filter = cv2.medianBlur(img, 9)

# Show image after median filtering
cv2.imshow('Median Filtered Image', img_median_filter)
cv2.waitKey(0)
cv2.destroyWindow('Median Filtered Image')

# Apply a bilateral filter to the image
img_bilateral_filter = cv2.bilateralFilter(img, 9, 75, 75)

# Show image after bilateral filtering
cv2.imshow('image_bilateral_filter', img_bilateral_filter)
cv2.waitKey(0)
cv2.destroyWindow('image_bilateral_filter')

Exercise 1.2: Add salt and pepper noise to a grayscale image and check the result of previous filters

In [56]:
img = cv2.imread(os.path.join(dataDir, 'coins_02.jpg'))

# Convert to grayscale
grayscale_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

h, w = grayscale_img.shape
num_pixels = h * w
num_pixels_to_alter = int(num_pixels * 0.05)

pixels = np.random.randint(num_pixels, size=num_pixels_to_alter)

# Get coordinates
x = pixels % w
y = pixels % h

color = np.random.choice([0,1], num_pixels_to_alter)

for i in range(num_pixels_to_alter):
    grayscale_img[y[i], x[i]] = color[i]*255

# Apply a median filter to the noisy image
img_median_filter = cv2.medianBlur(grayscale_img, 5)

# Apply a bilateral filter to the noisy image
img_bilateral_filter = cv2.bilateralFilter(grayscale_img, 9, 75, 75)

# Show the original noisy image
cv2.imshow('Noisy Image', grayscale_img)
cv2.waitKey(0)
cv2.destroyWindow('Noisy Image')

# Show the image after median filtering
cv2.imshow('Median Filter', img_median_filter)
cv2.waitKey(0)
cv2.destroyWindow('Median Filter')

# Show the image after bilateral filtering
cv2.imshow('Bilateral Filter', img_bilateral_filter)
cv2.waitKey(0)
cv2.destroyWindow('Bilateral Filter')

Exercise 1.3: Reproduce gaussian blur with custom convolution (using [ndimage.convolve()](https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.convolve.html))

In [66]:
from scipy import ndimage

img = cv2.imread(os.path.join(dataDir, 'coins_02.jpg'))

# Convert to grayscale
grayscale_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Normalize image
img_normalized = grayscale_img / 255

# Define Gaussian kernel
gaussian_kernel = np.array([[1,2,1],
                            [2,4,2],
                            [1,2,1]])

# Normalize kernel
gaussian_kernel = gaussian_kernel / np.sum(gaussian_kernel)

# Apply convolution
convolution_result = ndimage.convolve(img_normalized, gaussian_kernel)

# Apply Gaussian blur
gaussian_blur = cv2.GaussianBlur(grayscale_img, (3,3), 0)

# Show image
cv2.imshow('image', grayscale_img)
cv2.imshow('gaussian blur', gaussian_blur)
cv2.imshow('convolution', convolution_result)
cv2.waitKey(0)
cv2.destroyAllWindows()

Exercise 1.4: Define custom convolution (using [ndimage.convolve()](https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.convolve.html))

In [87]:
from scipy import ndimage

img = cv2.imread(os.path.join(dataDir, 'coins_02.jpg'))

# Convert to grayscale
grayscale_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Define a custom kernel for sharpening
custom_kernel = np.array([[0, -1, 0],
                          [-1, 5, -1],
                          [0, -1, 0]])

# Normalize kernel
custom_kernel = custom_kernel / np.sum(custom_kernel)

# Apply custom convolution
convolution_result = ndimage.convolve(grayscale_img, custom_kernel)

# Show images
cv2.imshow('Original Image', grayscale_img)
cv2.imshow('Custom Convolution Result', convolution_result)
cv2.waitKey(0)
cv2.destroyAllWindows()

Exercise 1.5: Add salt and pepper noise to a colored image and apply the previous filters 

In [105]:
# Load the image
img = cv2.imread(os.path.join(dataDir, 'coins_02.jpg'))

# Get image dimensions
h, w, _ = img.shape

# Calculate the number of pixels to alter
num_pixels = h * w
num_pixels_to_alter = int(num_pixels * 0.05)

# Generate random pixel coordinates
pixels = np.random.randint(num_pixels, size=num_pixels_to_alter)

# Get coordinates
x = pixels % w
y = pixels // w  # Note the correction here

# Generate random colors (0 or 255) for salt and pepper noise
colors = np.random.choice([0, 255], size=num_pixels_to_alter)

# Apply salt and pepper noise
for i in range(num_pixels_to_alter):
    img[y[i], x[i]] = [colors[i]] * 3  # Apply the same color to all channels

# Apply a median filter to the noisy image
img_median_filter = cv2.medianBlur(img, 5)

# Apply a bilateral filter to the noisy image
img_bilateral_filter = cv2.bilateralFilter(img, 9, 75, 75)

# Define a custom kernel for sharpening
custom_kernel = np.array([[0, -1, 0],
                          [-1, 5, -1],
                          [0, -1, 0]])

# Apply custom convolution to each color channel separately
convolution_result = np.zeros_like(img, dtype=np.float32)
for i in range(3):
    convolution_result[:, :, i] = ndimage.convolve(img[:, :, i].astype(np.float32), custom_kernel)

# Clip the pixel values to ensure they are within the valid range
convolution_result = np.clip(convolution_result, 0, 255)

# Convert the result back to uint8 for visualization
convolution_result_vis = convolution_result.astype(np.uint8)

# Show the original noisy image
cv2.imshow('Noisy Image', img)
cv2.waitKey(0)
cv2.destroyWindow('Noisy Image')

# Show the image after median filtering
cv2.imshow('Median Filter', img_median_filter)
cv2.waitKey(0)
cv2.destroyWindow('Median Filter')

# Show the image after bilateral filtering
cv2.imshow('Bilateral Filter', img_bilateral_filter)
cv2.waitKey(0)
cv2.destroyWindow('Bilateral Filter')

# Show the image after custom convolution
cv2.imshow('Custom Convolution Result', convolution_result_vis)
cv2.waitKey(0)
cv2.destroyAllWindows()

Exercise 1.6: Apply custom filter to colored image with noise

In [108]:
# Load the image
img = cv2.imread(os.path.join(dataDir, 'coins_02.jpg'))

# Get image dimensions
h, w, _ = img.shape

# Calculate the number of pixels to alter
num_pixels = h * w
num_pixels_to_alter = int(num_pixels * 0.05)

# Generate random pixel coordinates
pixels = np.random.randint(num_pixels, size=num_pixels_to_alter)

# Get coordinates
x = pixels % w
y = pixels // w

# Generate random colors (0 or 255) for salt and pepper noise
colors = np.random.choice([0, 255], size=num_pixels_to_alter)

# Apply salt and pepper noise
for i in range(num_pixels_to_alter):
    img[y[i], x[i]] = [colors[i]] * 3  # Apply the same color to all channels

# Define a custom kernel for custom filtering
custom_kernel = np.array([[0, -1, 0],
                          [-1, 5, -1],
                          [0, -1, 0]])

# Normalize kernel
custom_kernel = custom_kernel / np.sum(custom_kernel)

# Apply custom convolution to each color channel separately
filtered_image = np.zeros_like(img, dtype=np.float32)
for i in range(3):
    filtered_image[:, :, i] = ndimage.convolve(img[:, :, i].astype(np.float32), custom_kernel)

# Clip the pixel values to ensure they are within the valid range
filtered_image = np.clip(filtered_image, 0, 255)

# Convert the result back to uint8 for visualization
filtered_image = filtered_image.astype(np.uint8)

# Show the original noisy image
cv2.imshow('Noisy Image', img)
cv2.waitKey(0)
cv2.destroyWindow('Noisy Image')

# Show the image after custom filtering
cv2.imshow('Filtered Image', filtered_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 2. Histogram Equalization

In [126]:
# Load low contrast image
img = cv2.imread(os.path.join(dataDir, 'face_lowContrast_01.jpg'), 0) # Change this, according to your image's path

cv2.imshow('image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

[Histograms Equalization](https://docs.opencv.org/master/d6/dc7/group__imgproc__hist.html#ga7e54091f0c937d49bf84152a16f76d6e)

In [112]:
# Increasing contrast with Histograms Equalization
img_with_he = cv2.equalizeHist(img)

cv2.imshow('histogram_equalization', img_with_he)
cv2.waitKey(0)
cv2.destroyAllWindows()

[Contrast Limited Adaptive Histogram Equalization](https://docs.opencv.org/master/d6/dc7/group__imgproc__hist.html#gad689d2607b7b3889453804f414ab1018)

In [113]:
# Increasing contrast with CLAHE
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
img_with_CLAHE = clahe.apply(img)

cv2.imshow('clahe', img_with_CLAHE)
cv2.waitKey(0)
cv2.destroyAllWindows()

Exercise 2.1: Apply Histogram Equalization to a colored image

In [134]:
# Load the colored image
img = cv2.imread(os.path.join(dataDir, 'apple.jpg'))

# Split the image into channels
b, g, r = cv2.split(img)

# Apply histogram equalization to each channel
b_equalized = cv2.equalizeHist(b)
g_equalized = cv2.equalizeHist(g)
r_equalized = cv2.equalizeHist(r)

# Merge the equalized channels back into the image
img_equalized = cv2.merge((b_equalized, g_equalized, r_equalized))

# Display the original and equalized images
cv2.imshow('Original Image', img)
cv2.imshow('Equalized Image', img_equalized)
cv2.waitKey(0)
cv2.destroyAllWindows()

Exercise 2.2: Apply CLAHE to a colored image

In [135]:
# Load the colored image
img = cv2.imread(os.path.join(dataDir, 'apple.jpg'))

# Convert the image to LAB color space
lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)

# Split the LAB image into channels
l, a, b = cv2.split(lab)

# Apply CLAHE to the L channel
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
l_clahe = clahe.apply(l)

# Merge the CLAHE enhanced L channel with the original A and B channels
lab_clahe = cv2.merge((l_clahe, a, b))

# Convert the LAB image back to BGR color space
img_clahe = cv2.cvtColor(lab_clahe, cv2.COLOR_LAB2BGR)

# Display the original and CLAHE enhanced images
cv2.imshow('Original Image', img)
cv2.imshow('CLAHE Enhanced Image', img_clahe)
cv2.waitKey(0)
cv2.destroyAllWindows()